# Setup

In [12]:
# Import necessary packages
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException, TimeoutException
import pandas as pd
import time
from datetime import date, datetime
import math
from bs4 import BeautifulSoup
import urllib.request

# Google review url(s)
url = "https://www.google.com/maps/place/Nespresso/@49.2258269,-123.0014231,17z/data=!4m13!1m7!3m6!1s0x5486765957fb6f03:0x6b717a7e2475f0e8!2sNespresso!8m2!3d49.2258269!4d-122.9992344!10e1!3m4!1s0x5486765957fb6f03:0x6b717a7e2475f0e8!8m2!3d49.2258269!4d-122.9992344";

url_review = "https://www.google.com/maps/place/Nespresso/@49.2258269,-123.0014231,17z/data=!3m1!5s0x54867659512e6deb:0x8a75309114df0b12!4m15!1m7!3m6!1s0x5486765957fb6f03:0x6b717a7e2475f0e8!2sNespresso!8m2!3d49.2258269!4d-122.9992344!10e1!3m6!1s0x5486765957fb6f03:0x6b717a7e2475f0e8!8m2!3d49.2258269!4d-122.9992344!9m1!1b1";

# The date of running the web scraper
todayDate = str(datetime.now())[:-7];
print(f'Date & Time of Web Scraping: {todayDate}');

Date & Time of Web Scraping: 2023-01-29 20:55:10


# Run Web Scraper

In [13]:
# Instantiate web driver
driver = webdriver.Chrome("chromedriver");
driver.set_window_size(1024, 600);
driver.maximize_window();
time.sleep(5);

# Go to Google Reviews URL 
driver.get(url);
time.sleep(3);

In [14]:
# Get the total number of Google reviews and overall Google review rating
def get_generalMetrics(xpath, todayDate):
    driver.find_element("xpath", xpath).location_once_scrolled_into_view;
    time.sleep(1);
    metrics = driver.find_element("xpath", xpath).text.split("\n");
    data = [[metrics[0], metrics[1].split(" ")[0]]];
    df_generalMetrics = pd.DataFrame(data, columns=['Overall Rating','Total Number of Reviews']);
    df_generalMetrics['Webscraping Datetime'] = todayDate;
    return df_generalMetrics;

df_generalMetrics = get_generalMetrics(
    xpath = '//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[32]/div/div[2]', 
    todayDate = todayDate
);
df_generalMetrics

,Overall Rating,Total Number of Reviews,Webscraping Datetime
0,3.9,329,2023-01-29 20:55:10


In [15]:
# Distribution of stars given
def get_reviewStarDistribution(parent_xpath):
    # driver.find_element("xpath", parent_xpath).location_once_scrolled_into_view;
    data = [];
    for i in range(1,6):
        label = driver.find_element("xpath", parent_xpath+'tr['+str(i)+']').get_attribute("aria-label").split(", ");
        time.sleep(1);
        data.append([label[0].split(" ")[0], label[1].split(" ")[0]]);
    df_reviewStar = pd.DataFrame(data, columns=['Stars Given','Number of Reviews']);
    return df_reviewStar;

df_reviewStar = get_reviewStarDistribution('//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[32]/div/div[1]/table/tbody/');
df_reviewStar

,Stars Given,Number of Reviews
0,5,182
1,4,53
2,3,21
3,2,15
4,1,58


In [16]:
# Access full list of Google reviews
driver.get(url_review);
time.sleep(3);

# Sort reviews by "Newest"
driver.find_element("xpath", '//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[2]/div[7]/div[2]/button').click();

time.sleep(5);

driver.find_element("xpath", '//*[@id="action-menu"]/div[2]').click();

time.sleep(5);

In [17]:
# Scroll to the bottom of the page to get all reviews
def perform_scroll(scrollbar_xpath, y_pixel, number_limit):
    # Get scroll height
    last_height = driver.execute_script("return document.body.scrollHeight");
    time.sleep(5);
    number = 0;

    while True:
        number += 1;

        # Scroll down to bottom
        ele = driver.find_element("xpath", scrollbar_xpath);
        driver.execute_script(f'arguments[0].scrollBy(0, {y_pixel});', ele);

        # Wait to load page

        time.sleep(5);

        # Calculate new scroll height and compare with last scroll height
        print(f'Last Height: {last_height}');

        ele = driver.find_element("xpath", scrollbar_xpath);
        new_height = driver.execute_script("return arguments[0].scrollHeight", ele);

        print(f'New Height: {new_height}');

        if number == number_limit:
            break;

        if new_height == last_height:
            break;
        
        print("continue");
        last_height = new_height;

perform_scroll(
    scrollbar_xpath='//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[2]', 
    y_pixel=50000,
    number_limit=50
)

Last Height: 604
New Height: 9463
continue
Last Height: 9463
New Height: 14846
continue
Last Height: 14846
New Height: 18830
continue
Last Height: 18830
New Height: 23368
continue
Last Height: 23368
New Height: 27275
continue
Last Height: 27275
New Height: 30873
continue
Last Height: 30873
New Height: 33413
continue
Last Height: 33413
New Height: 35674
continue
Last Height: 35674
New Height: 37770
continue
Last Height: 37770
New Height: 39947
continue
Last Height: 39947
New Height: 42499
continue
Last Height: 42499
New Height: 44517
continue
Last Height: 44517
New Height: 46568
continue
Last Height: 46568
New Height: 48290
continue
Last Height: 48290
New Height: 50371
continue
Last Height: 50371
New Height: 52010
continue
Last Height: 52010
New Height: 53879
continue
Last Height: 53879
New Height: 56056
continue
Last Height: 56056
New Height: 58258
continue
Last Height: 58258
New Height: 60458
continue
Last Height: 60458
New Height: 62596
continue
Last Height: 62596
New Height: 65257
c

In [18]:
# Get the names of all reviewers
def get_namesOfReviewers(div_limit, reviews_xpath):
    i = 1;
    reviewers = [];
    while i <= div_limit:
        try:
            label = driver.find_element(By.XPATH, f'{reviews_xpath}/div[{str(i)}]').get_attribute("aria-label");
            if label != None:
                reviewers.append(label);
            i += 1;
        except NoSuchElementException:
            break;
    return reviewers;

reviewers = get_namesOfReviewers(
    div_limit = 1000, 
    reviews_xpath = '//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[2]/div[9]'
)

print(reviewers[len(reviewers)-1]);
print(len(reviewers));

George Gunay
329


In [19]:
# Click the 'More' button where the reviews are very long
button_list = driver.find_elements(By.TAG_NAME, 'button');
for button in button_list:
    if button.text == 'More':
        button.click();
        time.sleep(5);

In [ ]:
# //*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[2]/div[9]/div[985]

In [22]:
# Retrieve actual Google Reviews
def get_reviewData(reviewers, reviews_xpath, number_limit):
    data = [];
    for reviewer in reviewers:
        for i in range(1, number_limit+1):
            label = driver.find_element(By.XPATH, f'{reviews_xpath}/div[{str(i)}]');
            if label.get_attribute('aria-label') == reviewer:
                label_items = label.text.split("\n");
                row = [];
                count = 0;
                for item in label_items:
                    if item not in ['New','NEW','Like','Share']:
                        row.append(item);
                        count += 1;
                    if count == 4:
                        break;
                if len(row) == 3:
                    row.append(None);
                stars = driver.find_element(By.XPATH, f'{reviews_xpath}/div[{str(i)}]/div/div[3]/div[4]/div[1]/span[2]').get_attribute("aria-label");
                row.append(stars);
                print(row);
        data.append(row);
    df_reviewData = pd.DataFrame(data, columns=['Name','Total Reviews Given','Time of Review','Review','Stars Given']);
    df_reviewData = df_reviewData.drop_duplicates();
    return df_reviewData;

df_reviewData = get_reviewData(
    reviewers=reviewers, 
    reviews_xpath='//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[2]/div[9]', 
    number_limit=985
);
df_reviewData

['Vielka Duran', 'Local Guide · 41 reviews', '6 hours ago', None, ' 5 stars ']
['LASHLOVE33', '3 reviews', '2 weeks ago', 'Hit and miss. Staff are either extremely rude or extremely nice.', ' 2 stars ']
['Linda Wilke', '4 reviews', '4 weeks ago', "Tried calling this boutique to ask some questions about their machines and coffees, but wasn't able to connect directly with them by phone.  We took the risk and drove out from the Fraser Valley (50ish minutes).  We spent less than 5 minutes in the store, as the employee couldn't answer our questions about the different Vurtuo coffees (I was hoping for something similar in taste to the Fortisso lungo of the original line) and we weren't allowed to try any samples (instead we were directed to your downtown location).  Bring back the ability to try your coffee and bring back another 'pop up' further East.  Not all your customers live close to Vancouver/Burnaby.", ' 2 stars ']
['Alex Reyes', '2 reviews', 'a month ago', 'Response from the owner a

,Name,Total Reviews Given,Time of Review,Review,Stars Given
0,Vielka Duran,Local Guide · 41 reviews,6 hours ago,None,5 stars
1,LASHLOVE33,3 reviews,2 weeks ago,Hit and miss. Staff are either extremely rude ...,2 stars
2,Linda Wilke,4 reviews,4 weeks ago,Tried calling this boutique to ask some questi...,2 stars
3,Alex Reyes,2 reviews,a month ago,Response from the owner a month ago,3 stars
4,Hugo Nakamura,Local Guide · 25 reviews,a month ago,"EDIT: The second time coming to the store, the...",5 stars
...,...,...,...,...,...
324,JJ BGS,Local Guide · 55 reviews,4 years ago,We usually get our cups via amazon. We were in...,1 star
325,Jina Lee,4 years ago,1 star,None,None
326,Pranjali Viral T,Local Guide · 100 reviews,4 years ago,We are Nespresso club member for past few year...,5 stars
327,Kevin Lee,Local Guide · 151 reviews,4 years ago,Like another post said waited 30m. Was interes...,1 star


# Export the Scraped Data

In [23]:
rawScrapedData = pd.ExcelWriter("data/raw_data.xlsx");

df_generalMetrics.to_excel(rawScrapedData, sheet_name="General Metrics", index=False);

df_reviewStar.to_excel(rawScrapedData, sheet_name='Stars Distribution', index=False);

df_reviewData.to_excel(rawScrapedData, sheet_name='Reviews', index=False);

rawScrapedData.save();

C:\Users\kunal\AppData\Local\Temp/ipykernel_7208/2781667349.py:9: FutureWarning: save is not part of the public API, usage can give in unexpected results and will be removed in a future version
  rawScrapedData.save();


In [24]:
df_generalMetrics.to_csv('data/rawData_GeneralMetrics.csv', index=False);

df_reviewStar.to_csv('data/rawData_StarsDistribution.csv', index=False);

df_reviewData.to_csv('data/rawData_Reviews.csv', index=False);

# Shutdown Web Scraper

In [25]:
driver.close()